# Predicting Re-Opening Success

In [1]:
import itertools

from matplotlib.pylab import plt
from matplotlib.ticker import FuncFormatter
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from matplotlib import pyplot, lines
from matplotlib.patches import Patch
import matplotlib

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import RFECV

from sklearn.metrics import confusion_matrix

from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS



from scipy.stats import ttest_ind

import researchpy as rpy

from loguru import logger



import pandas as pd
import numpy as np

import seaborn as sns

import datetime 


%matplotlib inline

# Parameters

In [2]:
ctp_peak_model_dataset = '../data/processed/peak_model_ctp_covid.csv'
owid_peak_model_dataset = '../data/processed/peak_model_owid_covid.csv'

lockdown_dataset = '../data/expanded_lockdowns.pkl'

# Load and Filter the Datasets

In [3]:
logger.info('Loading CTP peak models @ %s' % ctp_peak_model_dataset)

ctp = pd.read_csv(ctp_peak_model_dataset, parse_dates=['date'], low_memory=False)

ctp['region'] = 'us'

ctp.shape, ctp.state.nunique(), ctp['date'].max(), ctp.columns

2020-07-28 14:26:29.011 | INFO     | __main__:<module>:1 - Loading CTP peak models @ ../data/processed/peak_model_ctp_covid.csv


((7599, 22),
 51,
 Timestamp('2020-07-27 00:00:00'),
 Index(['aggregation', 'date', 'google_mobility_level_rolling_mean_stage',
        'new_deaths_rolling_mean_stage', 'new_cases_rolling_mean_stage',
        'state', 'new_cases', 'new_deaths', 'population',
        'google_mobility_level', 'google_mobility_level_by_pop',
        'new_cases_rolling_mean', 'new_deaths_rolling_mean',
        'google_mobility_level_rolling_mean', 'new_cases_rolling_mean_rel',
        'new_deaths_rolling_mean_rel', 'google_mobility_level_rolling_mean_rel',
        'new_cases_rolling_mean_stage_day', 'new_deaths_rolling_mean_stage_day',
        'google_mobility_level_rolling_mean_stage_day',
        'new_cases_rolling_mean_cummax', 'region'],
       dtype='object'))

In [4]:
logger.info('Loading OWID peak models @ %s' % ctp_peak_model_dataset)

owid = pd.read_csv(owid_peak_model_dataset, parse_dates=['date']).rename(columns={'country':'state'})

owid.shape, owid.state.nunique(), owid.date.max(), owid.columns

2020-07-28 14:26:29.190 | INFO     | __main__:<module>:1 - Loading OWID peak models @ ../data/processed/peak_model_ctp_covid.csv


((38133, 25),
 207,
 Timestamp('2020-07-28 00:00:00'),
 Index(['aggregation', 'date', 'google_mobility_level_rolling_mean_stage',
        'new_deaths_rolling_mean_stage', 'new_cases_rolling_mean_stage',
        'country_id', 'state', 'country_code', 'continent', 'region',
        'new_cases', 'new_deaths', 'population', 'google_mobility_level',
        'google_mobility_level_by_pop', 'new_cases_rolling_mean',
        'new_deaths_rolling_mean', 'google_mobility_level_rolling_mean',
        'new_cases_rolling_mean_rel', 'new_deaths_rolling_mean_rel',
        'google_mobility_level_rolling_mean_rel',
        'new_cases_rolling_mean_stage_day', 'new_deaths_rolling_mean_stage_day',
        'google_mobility_level_rolling_mean_stage_day',
        'new_cases_rolling_mean_cummax'],
       dtype='object'))

In [5]:
# with_country_codes = owid['country_code'].notnull()
# owid_with_country_codes = owid[with_country_codes]

# owid_with_country_codes.shape, owid_with_country_codes['state'].nunique()

## Combine Countries and US Datasets

In [6]:
# Rename country as state and combine.
df = pd.concat([owid.rename(columns={'country': 'state'}), ctp], ignore_index=True, sort=False).sort_values(by=['state', 'date'])
df.shape, df.state.nunique(), df.aggregation.nunique()

((45732, 25), 258, 287)

## Eliminate aggregations without mobility data

In [7]:
with_mobility_data = df.groupby('aggregation').apply(lambda g: g['google_mobility_level_rolling_mean'].any())

df = df[df['aggregation'].isin(with_mobility_data[with_mobility_data].index)]

df.shape, df['state'].nunique()

((35418, 25), 183)

## Focus on country/state level

In [8]:
# df = df[df['aggregation']!=df['region']]
# df.shape, df['state'].nunique()

# Lockdown Features
The key lockdown features include the following mobility and case/death related features.


Mobility related features:
* lockdown_start_date
* rebound_start_date

* lockdown_min_mobility_date
* lockdown_min_mobility_level

* lockdown_mean_mobility_level
* rebound_mean_mobility_level

Cases/Deaths related features:
* lockdown_entry_level_cases
* lockdown_exit_level_cases

* lockdown_peak_date_cases
* lockdown_peak_value_cases
* lockdown_peak_level_cases

* overall_peak_date_cases
* overall_peak_value_cases




In [9]:
df = df.sort_values(by='date')

## Extracting the basic features 

In [10]:
def extract_lockdown_features(location, region, population, cases_data, deaths_data, mobility_data, mobility_stages):
    """Extract a full set of lockdown and rebound features for a given state/country/aggregation
    and return as a series."""
    
    # Total cases and deaths
    total_cases = cases_data.sum()
    total_deaths = deaths_data.sum()
    cfr = total_deaths/total_cases
    
    # The overall start/end dates for which we have case data
    overall_start_date = cases_data[cases_data>0].index[0]
    overall_end_date = cases_data.index[-1]
    
    # Current mobility/growth levels
    current_mobility_level = mobility_data.dropna().iloc[-1]
    current_value_cases = cases_data.dropna().iloc[-1]
    current_value_deaths = deaths_data.dropna().iloc[-1]

    # Mobility features
    lockdown_start_date = mobility_stages[mobility_stages=='D'].index[0]  # First day of the Drop
    rebound_start_date = mobility_stages[mobility_stages=='B'].index[0]   # First day of the Bounce
        
    
    lockdown_duration_days = (rebound_start_date-lockdown_start_date).days
    rebound_duration_days = (overall_end_date-rebound_start_date).days

    # Mobility during lockdown
    lockdown_mobility = mobility_data.loc[lockdown_start_date:rebound_start_date]
    lockdown_min_mobility_date = lockdown_mobility.idxmin()
    lockdown_min_mobility_level = lockdown_mobility.min()
    lockdown_mean_mobility_level = lockdown_mobility.mean()

    # Mobility during rebound
    rebound_mobility = mobility_data.loc[rebound_start_date:]
    rebound_min_mobility_date = rebound_mobility.idxmin()
    rebound_min_mobility_level = rebound_mobility.min()
    rebound_mean_mobility_level = rebound_mobility.mean()

    
    # Cases/Deaths

    # Overall features
    overall_peak_date_cases = cases_data.idxmax()
    overall_peak_value_cases = cases_data.max()

    overall_peak_date_deaths = deaths_data.idxmax()
    overall_peak_value_deaths = deaths_data.max()

    # Lockown features - cases; note levels measure wrt overall peak
    lockdown_cases = cases_data.loc[lockdown_start_date:rebound_start_date]
    
    lockdown_peak_date_cases = lockdown_cases.idxmax()
    lockdown_peak_value_cases = lockdown_cases.max()
    lockdown_peak_level_cases = lockdown_peak_value_cases/overall_peak_value_cases
    
    lockdown_mean_value_cases = lockdown_cases.mean()
    lockdown_mean_level_cases = lockdown_mean_value_cases/overall_peak_value_cases

    # Lockdown features - deaths; note levels measure wrt overall peak
    lockdown_deaths = deaths_data.loc[lockdown_start_date:rebound_start_date]
    
    lockdown_peak_date_deaths = lockdown_deaths.idxmax()
    lockdown_peak_value_deaths = lockdown_deaths.max()
    lockdown_peak_level_deaths = lockdown_peak_value_deaths/overall_peak_value_deaths
    
    lockdown_mean_value_deaths = lockdown_deaths.mean()
    lockdown_mean_level_deaths = lockdown_mean_value_deaths/overall_peak_value_deaths
    
    # Days to/from lockdown peak
    lockdown_days_to_peak_cases = (lockdown_peak_date_cases-lockdown_start_date).days
    lockdown_days_to_peak_deaths = (lockdown_peak_date_deaths-lockdown_start_date).days

    lockdown_days_from_peak_cases = (rebound_start_date-lockdown_peak_date_cases).days
    lockdown_days_from_peak_deaths = (rebound_start_date-lockdown_peak_date_deaths).days

    
    # Entry/Exit Features
    # Note the entry/exit levels are wrt the lockdown peak values
    lockdown_entry_value_cases = cases_data.loc[lockdown_start_date]
    lockdown_entry_level_cases = lockdown_entry_value_cases/lockdown_peak_value_cases
    
    lockdown_entry_value_deaths = cases_data.loc[lockdown_start_date]
    lockdown_entry_level_deaths = lockdown_entry_value_deaths/lockdown_peak_value_deaths

    lockdown_exit_value_cases = cases_data.loc[rebound_start_date]
    lockdown_exit_level_cases = lockdown_exit_value_cases/lockdown_peak_value_cases
    
    lockdown_exit_value_deaths = deaths_data.loc[rebound_start_date]
    lockdown_exit_level_deaths = lockdown_exit_value_deaths/lockdown_peak_value_deaths
    
    lockdown_entry_exit_ratio_cases = lockdown_exit_value_cases/lockdown_entry_value_cases
    lockdown_entry_exit_ratio_deaths = lockdown_exit_value_deaths/lockdown_entry_value_deaths

    
    
    # Cases/deaths during rebound
    rebound_cases = cases_data[rebound_start_date:]
    rebound_deaths = deaths_data[rebound_start_date:]
    
    rebound_mean_value_cases = rebound_cases.mean()
    rebound_mean_level_cases = rebound_mean_value_cases/overall_peak_value_cases
    
    rebound_mean_value_deaths = rebound_deaths.mean()
    rebound_mean_level_deaths = rebound_mean_value_deaths/overall_peak_value_deaths
    
    return pd.Series(dict(
        
        location=location,
        region=region,
        population=population, 
        
        total_cases=total_cases,
        total_deaths=total_deaths,
        cfr=cfr,

        
        overall_start_date=overall_start_date,
        overall_end_date=overall_end_date,
        
        current_mobility_level=current_mobility_level,
        current_value_cases=current_value_cases,
        current_value_deaths=current_value_deaths,
        
        lockdown_start_date=lockdown_start_date,
        rebound_start_date=rebound_start_date,
        
        lockdown_duration_days=lockdown_duration_days,
        rebound_duration_days=rebound_duration_days,
        
        lockdown_min_mobility_date=lockdown_min_mobility_date,
        lockdown_min_mobility_level=lockdown_min_mobility_level,
        lockdown_mean_mobility_level=lockdown_mean_mobility_level,

        rebound_min_mobility_date=rebound_min_mobility_date,
        rebound_min_mobility_level=rebound_min_mobility_level,
        rebound_mean_mobility_level=rebound_mean_mobility_level,
        
        overall_peak_date_cases=overall_peak_date_cases,
        overall_peak_value_cases=overall_peak_value_cases,
        overall_peak_date_deaths=overall_peak_date_deaths,
        overall_peak_value_deaths=overall_peak_value_deaths,
        
        lockdown_days_to_peak_cases=lockdown_days_to_peak_cases,
        lockdown_days_to_peak_deaths=lockdown_days_to_peak_deaths,
        lockdown_days_from_peak_cases=lockdown_days_from_peak_cases,
        lockdown_days_from_peak_deaths=lockdown_days_from_peak_deaths,
        
        lockdown_entry_value_cases=lockdown_entry_value_cases,
        lockdown_exit_value_cases=lockdown_exit_value_cases,
        lockdown_entry_value_deaths=lockdown_entry_value_deaths,
        lockdown_exit_value_deaths=lockdown_exit_value_deaths,
        
        lockdown_entry_level_cases=lockdown_entry_level_cases,
        lockdown_entry_level_deaths=lockdown_entry_level_deaths,
        lockdown_exit_level_cases=lockdown_exit_level_cases,
        lockdown_exit_level_deaths=lockdown_exit_level_deaths,
        
        lockdown_entry_exit_ratio_cases=lockdown_entry_exit_ratio_cases,
        lockdown_entry_exit_ratio_deaths=lockdown_entry_exit_ratio_deaths,

        lockdown_peak_date_cases=lockdown_peak_date_cases,
        lockdown_peak_value_cases=lockdown_peak_value_cases,
        lockdown_peak_level_cases=lockdown_peak_level_cases,

        lockdown_peak_date_deaths=lockdown_peak_date_deaths,
        lockdown_peak_value_deaths=lockdown_peak_value_deaths,
        lockdown_peak_level_deaths=lockdown_peak_level_deaths,
        
        lockdown_mean_value_cases=lockdown_mean_value_cases,
        lockdown_mean_level_cases=lockdown_mean_level_cases,
        
        lockdown_mean_value_deaths=lockdown_mean_value_deaths,
        lockdown_mean_level_deaths=lockdown_mean_level_deaths,
        
        rebound_mean_value_cases=rebound_mean_value_cases,
        rebound_mean_level_cases=rebound_mean_level_cases,
        rebound_mean_value_deaths=rebound_mean_value_deaths,
        rebound_mean_level_deaths=rebound_mean_level_deaths,
    )).fillna(0)



## Additional Derived Features

In [11]:
def gen_derived_features(lockdowns):

    lockdowns['lockdown_mean_value_cases_per_million'] = 1000000*lockdowns['lockdown_mean_value_cases']/lockdowns['population']
    lockdowns['lockdown_peak_value_cases_per_million'] = 1000000*lockdowns['lockdown_peak_value_cases']/lockdowns['population']

    lockdowns['lockdown_mean_value_deaths_per_100k'] = 10000000*lockdowns['lockdown_mean_value_deaths']/lockdowns['population']
    lockdowns['lockdown_peak_value_deaths_per_100k'] = 10000000*lockdowns['lockdown_peak_value_deaths']/lockdowns['population']

    lockdowns['rebound_mean_value_cases_per_million'] = 1000000*lockdowns['rebound_mean_value_cases']/lockdowns['population']
    lockdowns['rebound_mean_value_deaths_per_100k'] = 10000000*lockdowns['rebound_mean_value_deaths']/lockdowns['population']

    lockdowns['rebound_ratio_cases'] = lockdowns['rebound_mean_value_cases']/lockdowns['lockdown_mean_value_cases']
    lockdowns['rebound_ratio_deaths'] = lockdowns['rebound_mean_value_deaths']/lockdowns['lockdown_mean_value_deaths']
    lockdowns['rebound_ratio_mobility_level'] = lockdowns['rebound_mean_mobility_level']/lockdowns['lockdown_mean_mobility_level']

    lockdowns['is_increasing_rebound'] = lockdowns['rebound_ratio_cases']>1

    strong_threshold, conservative_threshold = lockdowns['lockdown_mean_mobility_level'].median(), lockdowns['rebound_mean_mobility_level'].median()
    lockdowns['is_strong_lockdown'] = lockdowns['lockdown_mean_mobility_level'] < strong_threshold
    lockdowns['is_conservative_rebound'] = lockdowns['rebound_mean_mobility_level'] < conservative_threshold

    lockdowns['is_successful_rebound'] = lockdowns['rebound_ratio_cases']<1

    lockdowns['is_successful_rebound_deaths'] = lockdowns['rebound_ratio_deaths']<1

    lockdowns = lockdowns.replace(np.inf, np.nan)
    lockdowns = lockdowns.dropna()
    
    return lockdowns

## Gen the lockdown df

In [12]:
def gen_lockdowns(df, to_date=datetime.datetime.today()):
    
    # Focus on data only up to the to_date
    up_to_date = df['date']<=to_date
    df = df[up_to_date]
    
    # Focus on states that have D, H, B stages
    valid_lockdowns = df.groupby('aggregation').apply(
        lambda g: set(['D', 'H', 'B']).difference(set(g['google_mobility_level_rolling_mean_stage'].unique())) == set())
    valid_lockdowns = valid_lockdowns[valid_lockdowns].index

    rebounding = df['aggregation'].isin(valid_lockdowns)

    # The basic features
    lockdowns = df[rebounding].set_index('date').groupby('aggregation').apply(
        lambda g: extract_lockdown_features(
            g.name, g['region'].unique()[0], g['population'].max(),
            g['new_cases_rolling_mean'], g['new_deaths_rolling_mean'],
            g['google_mobility_level_rolling_mean_rel'], g['google_mobility_level_rolling_mean_stage'],
        )
    )
    
    # The additional features
    lockdowns = gen_derived_features(lockdowns)
        
    return lockdowns
    

In [13]:
# Generate a new lockdown dataframe for each date and combine to produce a single large extanded lockdown dataframe
def expand_lockdowns(df, from_date, to_date):
    num_days = (to_date-from_date).days
    return pd.concat([gen_lockdowns(df, to_date=from_date+datetime.timedelta(days=days)) for days in range(num_days)])


In [14]:
today = datetime.datetime.today()
from_date = today - datetime.timedelta(weeks=12)
 
expanded_lockdowns = expand_lockdowns(df, from_date, today)
expanded_lockdowns.shape

/home/barry/anaconda3/envs/covid-19/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in double_scalars
/home/barry/anaconda3/envs/covid-19/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in double_scalars
/home/barry/anaconda3/envs/covid-19/lib/python3.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in double_scalars
/home/barry/anaconda3/envs/covid-19/lib/python3.7/site-packages/ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in double_scalars
/home/barry/anaconda3/envs/covid-19/lib/python3.7/site-packages/ipykernel_launcher.py:104: RuntimeWarning: invalid value encountered in double_scalars


/home/barry/anaconda3/envs/covid-19/lib/python3.7/site-packages/ipykernel_launcher.py:91: RuntimeWarning: divide by zero encountered in double_scalars
/home/barry/anaconda3/envs/covid-19/lib/python3.7/site-packages/ipykernel_launcher.py:92: RuntimeWarning: divide by zero encountered in double_scalars


/home/barry/anaconda3/envs/covid-19/lib/python3.7/site-packages/ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in double_scalars


(15534, 67)

# Save Lockdown Data

In [15]:
logger.info('Saving expanded lockdown data -> %s' % lockdown_dataset)

2020-07-28 14:28:22.040 | INFO     | __main__:<module>:1 - Saving expanded lockdown data -> ../data/expanded_lockdowns.pkl


In [16]:
expanded_lockdowns.reset_index().to_pickle(lockdown_dataset)
expanded_lockdowns.reset_index().shape

(15534, 68)

In [17]:
expanded_lockdowns.loc['eu']

,location,region,population,total_cases,total_deaths,cfr,overall_start_date,overall_end_date,current_mobility_level,current_value_cases,...,rebound_mean_value_cases_per_million,rebound_mean_value_deaths_per_100k,rebound_ratio_cases,rebound_ratio_deaths,rebound_ratio_mobility_level,is_increasing_rebound,is_strong_lockdown,is_conservative_rebound,is_successful_rebound,is_successful_rebound_deaths
aggregation,,,,,,,,,,,,,,,,,,,,,
eu,eu,eu,444919060.0,9.873561e+05,110809.714286,0.112229,2020-01-25,2020-05-09,0.504781,7366.714286,...,16.557426,26.235912,0.411778,0.566368,1.024791,False,True,True,True,True
eu,eu,eu,444919060.0,9.949356e+05,111913.857143,0.112484,2020-01-25,2020-05-10,0.512438,7579.428571,...,16.796474,25.526312,0.417723,0.551049,1.032563,False,True,True,True,True
eu,eu,eu,444919060.0,1.002300e+06,112978.857143,0.112720,2020-01-25,2020-05-11,0.518803,7364.571429,...,16.715186,24.996521,0.415701,0.539613,1.039461,False,True,True,True,True
eu,eu,eu,444919060.0,1.009471e+06,114010.714286,0.112941,2020-01-25,2020-05-12,0.525912,7170.714286,...,16.565613,24.545395,0.411982,0.529874,1.046519,False,True,True,True,True
eu,eu,eu,444919060.0,1.016479e+06,115006.000000,0.113142,2020-01-25,2020-05-13,0.531831,7007.714286,...,16.402598,24.110324,0.407928,0.520482,1.053156,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eu,eu,eu,444919060.0,1.323903e+06,135067.571429,0.102022,2020-01-25,2020-07-23,0.713296,6009.428571,...,10.170782,7.519151,0.252944,0.162320,1.312393,False,True,True,True,True
eu,eu,eu,444919060.0,1.330241e+06,135149.428571,0.101598,2020-01-25,2020-07-24,0.713296,6338.428571,...,10.223710,7.445393,0.254260,0.160727,1.312393,False,True,True,True,True
eu,eu,eu,444919060.0,1.336739e+06,135231.571429,0.101165,2020-01-25,2020-07-25,0.713296,6498.000000,...,10.279879,7.373609,0.255657,0.159178,1.312393,False,True,True,True,True
